# 6.5 梯度下降法的向量化

In [2]:
import numpy as np
from sklearn import datasets

In [9]:
boston  = datasets.load_boston()
X = boston.data
y = boston.target

X = X[y < 50.0]
y = y[y < 50.0]

In [10]:
from playML.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, seed=666)

In [14]:
from playML.LinearRegression import LinearRegression

lin_reg1 = LinearRegression()
%time lin_reg1.fit_normal(X_train, y_train)
lin_reg1.score(X_test, y_test)

Wall time: 152 ms


0.8129794056212832

## 6.5.1 使用梯度下降法

In [16]:
lin_reg2 = LinearRegression()
# 出现overflow Warning
lin_reg2.fit_gd(X_train, y_train)

D:\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\Administrator\Documents\PycharmProjects\Machine-Learning-Algorithms\06_Gradient_Descent\05_Vectorize-Gradient-Descent\playML\LinearRegression.py:33: RuntimeWarning: overflow encountered in square
  return np.sum((y - X_b.dot(theta)) ** 2) / len(y)
C:\Users\Administrator\Documents\PycharmProjects\Machine-Learning-Algorithms\06_Gradient_Descent\05_Vectorize-Gradient-Descent\playML\LinearRegression.py:54: RuntimeWarning: invalid value encountered in double_scalars
  if (abs(J(theta, X_b, y) - J(last_theta, X_b, y)) < epsilon):


LinearRegression()

In [18]:
# overflow, 真实的数据集中，每一个特征对应的规模是不一样的
lin_reg2.coef_

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [20]:
X_train[:5]

array([[1.42362e+01, 0.00000e+00, 1.81000e+01, 0.00000e+00, 6.93000e-01,
        6.34300e+00, 1.00000e+02, 1.57410e+00, 2.40000e+01, 6.66000e+02,
        2.02000e+01, 3.96900e+02, 2.03200e+01],
       [3.67822e+00, 0.00000e+00, 1.81000e+01, 0.00000e+00, 7.70000e-01,
        5.36200e+00, 9.62000e+01, 2.10360e+00, 2.40000e+01, 6.66000e+02,
        2.02000e+01, 3.80790e+02, 1.01900e+01],
       [1.04690e-01, 4.00000e+01, 6.41000e+00, 1.00000e+00, 4.47000e-01,
        7.26700e+00, 4.90000e+01, 4.78720e+00, 4.00000e+00, 2.54000e+02,
        1.76000e+01, 3.89250e+02, 6.05000e+00],
       [1.15172e+00, 0.00000e+00, 8.14000e+00, 0.00000e+00, 5.38000e-01,
        5.70100e+00, 9.50000e+01, 3.78720e+00, 4.00000e+00, 3.07000e+02,
        2.10000e+01, 3.58770e+02, 1.83500e+01],
       [6.58800e-02, 0.00000e+00, 2.46000e+00, 0.00000e+00, 4.88000e-01,
        7.76500e+00, 8.33000e+01, 2.74100e+00, 3.00000e+00, 1.93000e+02,
        1.78000e+01, 3.95560e+02, 7.56000e+00]])

In [21]:
lin_reg2.fit_gd(X_train, y_train, eta=0.000001)

LinearRegression()

In [22]:
# R2值很小，说明梯度下降法还没找到损失函数最小的位置
lin_reg2.score(X_test, y_test)

0.27586818724477224

In [23]:
# 加大搜寻的次数，看能否找到
%time lin_reg2.fit_gd(X_train, y_train, eta=0.000001, n_iters=1e6)

Wall time: 1min 7s


LinearRegression()

In [25]:
# 其实还是没有找到
lin_reg2.score(X_test, y_test)

0.7542932581943915

## 6.5.2 使用梯度下降法前进行数据归一化

In [26]:
from sklearn.preprocessing import StandardScaler

In [27]:
standardScaler = StandardScaler()
standardScaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [28]:
# 获取归一化后的train数据
X_train_standard = standardScaler.transform(X_train)

In [29]:
# 归一化以后，使用梯度下降法的速度明显提高了
lin_reg3 = LinearRegression()
%time lin_reg3.fit_gd(X_train_standard, y_train)

Wall time: 387 ms


LinearRegression()

In [30]:
X_test_standard = standardScaler.transform(X_test)

In [31]:
# R2的值与正规方程的值是一致的，说明也找到了损失函数的最小值。
lin_reg3.score(X_test_standard, y_test)

0.8129873310487505

## 6.5.3 梯度下降法的优势

In [33]:
# 构造一个 m个样本，n个特征的数据
m = 1000
n = 5000

big_X = np.random.normal(size=(m, n))

true_theta = np.random.uniform(0.0, 100.0, size=n+1)
# 乘上一个coef，再加上intercept，最后加上一个正态分布的噪音
big_y = big_X.dot(true_theta[1:]) + true_theta[0] + np.random.normal(0., 10., size=m)


In [34]:
# 正规方程对m*n的矩阵有很多的乘法运算，所有矩阵很大时，耗时会很长
big_reg1 = LinearRegression()
%time big_reg1.fit_normal(big_X, big_y)

Wall time: 12.9 s


LinearRegression()

In [35]:
# 
big_reg2 = LinearRegression()
%time big_reg2.fit_gd(big_X, big_y)

Wall time: 3.68 s


LinearRegression()